In [1]:
# Import required modules and initialize the builder from open_deep_research
import uuid 
import os, getpass
import open_deep_research   
print(open_deep_research.__version__) 
from IPython.display import Image, display, Markdown
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver
from open_deep_research.graph import builder
import requests
import json
from typing import Dict, Any, List

0.0.15


In [2]:
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [3]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("OPENAI_API_KEY")
# _set_env("ANTHROPIC_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("GROQ_API_KEY")
# _set_env("PERPLEXITY_API_KEY")

In [4]:
def get_search_results(query: str, max_results: int = 3) -> List[Dict[str, Any]]:
    """Get search results from Tavily API directly"""
    tavily_api_key = os.environ.get("TAVILY_API_KEY")
    if not tavily_api_key:
        raise ValueError("TAVILY_API_KEY is not set in environment variables")
        
    search_url = "https://api.tavily.com/search"
    search_params = {
        "api_key": tavily_api_key,
        "query": query,
        "search_depth": "basic",
        "max_results": max_results
    }
    
    try:
        search_response = requests.post(search_url, json=search_params)
        search_response.raise_for_status()  # Raise exception for HTTP errors
        search_data = search_response.json()
        
        if "results" in search_data:
            return search_data["results"]
        else:
            print(f"Unexpected search response format: {search_data}")
            return []
    except requests.exceptions.RequestException as e:
        print(f"Search API error: {str(e)}")
        return []


In [5]:
def generate_report(search_results: List[Dict[str, Any]], topic: str) -> str:
    """Generate report using Groq API directly"""
    groq_api_key = os.environ.get("GROQ_API_KEY")
    if not groq_api_key:
        raise ValueError("GROQ_API_KEY is not set in environment variables")
        
    groq_url = "https://api.groq.com/openai/v1/chat/completions"
    groq_headers = {
        "Authorization": f"Bearer {groq_api_key}",
        "Content-Type": "application/json"
    }
    
    # Format search results for the prompt
    formatted_results = ""
    for i, result in enumerate(search_results, 1):
        title = result.get("title", "Untitled Source")
        url = result.get("url", "No URL provided")
        content_snippet = result.get("content", "")[:200]  # First 200 chars of content
        formatted_results += f"Source {i}: {title}\nURL: {url}\nExcerpt: {content_snippet}...\n\n"
    
    # Create a more structured prompt
    prompt = f"""Based on these search results about {topic}, create a concise report following this structure:

1. Introduction (20 sentences)
   - Brief overview of {topic}

2. Key Concepts (34 bullet points)
   - Main ideas related to {topic}

3. Applications or Implications (23 sentences)
   - How {topic} is being used or its importance

4. Summary Table or List
   - A concise summary of the main points

SEARCH RESULTS:
{formatted_results}

Keep your total response under 300 lines."""
    
    groq_data = {
        "model": "meta-llama/llama-4-scout-17b-16e-instruct",  # Updated to use a more capable model
        "messages": [
            {"role": "system", "content": "You create concise, factual reports based on provided search results."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 1000,
        "temperature": 0.3
    }
    
    try:
        groq_response = requests.post(groq_url, headers=groq_headers, json=groq_data)
        groq_response.raise_for_status()  # Raise exception for HTTP errors
        groq_data = groq_response.json()
        
        if "choices" in groq_data and len(groq_data["choices"]) > 0:
            result = groq_data["choices"][0]["message"]["content"]
            return result
        else:
            error_msg = f"Unexpected Groq API response: {groq_data}"
            print(error_msg)
            return error_msg
    except requests.exceptions.RequestException as e:
        error_msg = f"Groq API error: {str(e)}"
        print(error_msg)
        return error_msg

In [6]:

# def set_env(var: str):
#     if not os.environ.get(var):
#         os.environ[var] = getpass.getpass(f"{var}: ")

# # Set necessary API keys
# set_env("TAVILY_API_KEY")
# set_env("GROQ_API_KEY")

# Main execution function
def run_research(topic: str, search_query: str = None):
    """Run the research pipeline for a given topic"""
    if search_query is None:
        search_query = f"{topic} explanation concepts applications"
        
    print(f"Searching for information about '{topic}'...")
    search_results = get_search_results(search_query, max_results=3)

    if search_results:
        print(f"Found {len(search_results)} search results. Generating report...")
        report = generate_report(search_results, topic)
        display(Markdown(f"# Report on {topic}\n\n{report}"))
        
        # Display sources
        sources_md = "## Sources\n\n"
        for i, result in enumerate(search_results, 1):
            title = result.get("title", "Untitled")
            url = result.get("url", "#")
            sources_md += f"{i}. [{title}]({url})\n"
        
        display(Markdown(sources_md))
    else:
        print("No search results found. Please check your Tavily API key and try again.")


# Example usage
if __name__ == "__main__":
    run_research("Gredient Ascent")

Searching for information about 'Gredient Ascent'...
Found 3 search results. Generating report...


# Report on Gredient Ascent

**Introduction**

Gradient Ascent is an optimization algorithm used to find the local maximum of a function. It is a variation of the Gradient Descent algorithm, but instead of minimizing a function, it maximizes it. The algorithm iteratively updates the parameters of the function in the direction of the gradient, which points towards the local maximum. Gradient Ascent is used in various machine learning applications, particularly in optimization problems. The concept of Gradient Ascent is essential in understanding its applications and differences with Gradient Descent. Gradient Ascent is also known as Gradient Ascension. The algorithm is simple to implement and can be used for both linear and non-linear functions. Gradient Ascent has various applications in data science and machine learning. The algorithm is based on the concept of gradients and derivatives. The gradient of a function represents the rate of change of the function with respect to its parameters. Gradient Ascent uses this gradient to update the parameters of the function. The algorithm is widely used in various fields, including computer science, mathematics, and engineering. Gradient Ascent has several advantages, including simplicity and efficiency. However, it also has some limitations, such as getting stuck in local maxima. Despite these limitations, Gradient Ascent remains a popular optimization algorithm. The algorithm can be used for both convex and non-convex functions. Gradient Ascent is an essential tool for data scientists and machine learning engineers. The algorithm has various applications in real-world problems. Gradient Ascent is used in optimization problems, such as maximizing a likelihood function. The algorithm is also used in linear regression and other machine learning models.

**Key Concepts**

* Gradient Ascent is an optimization algorithm
* Used to find the local maximum of a function
* Variation of Gradient Descent algorithm
* Updates parameters in the direction of the gradient
* Gradient points towards the local maximum
* Used in machine learning applications
* Particularly in optimization problems
* Simple to implement
* Can be used for linear and non-linear functions
* Based on gradients and derivatives
* Gradient represents rate of change of the function
* Updates parameters using the gradient
* Widely used in computer science, mathematics, and engineering
* Has several advantages, including simplicity and efficiency
* Can get stuck in local maxima
* Used for convex and non-convex functions
* Essential tool for data scientists and machine learning engineers
* Applications in real-world problems
* Used in optimization problems, such as maximizing a likelihood function
* Used in linear regression and other machine learning models
* Takes steps proportional to the gradient
* Approaches a local maximum of the function
* Difference with Gradient Descent: direction of update
* Gradient Ascent: maximize a function
* Gradient Descent: minimize a function

**Applications or Implications**

Gradient Ascent has various applications in machine learning and data science. It is used in optimization problems, such as maximizing a likelihood function. The algorithm is also used in linear regression and other machine learning models. Gradient Ascent is used in real-world problems, such as computer vision and natural language processing. The algorithm has several implications, including simplicity and efficiency. However, it also has some limitations, such as getting stuck in local maxima. Despite these limitations, Gradient Ascent remains a popular optimization algorithm. The algorithm can be used for both convex and non-convex functions. Gradient Ascent is an essential tool for data scientists and machine learning engineers. The algorithm has various applications in fields, including computer science, mathematics, and engineering. Gradient Ascent is used in various machine learning applications, particularly in optimization problems. The algorithm is widely used due to its simplicity and efficiency. Gradient Ascent has several applications in data analysis and machine learning. The algorithm is used in various industries, including finance and healthcare. Gradient Ascent has several implications for data scientists and machine learning engineers.

**Summary Table or List**

| **Concept** | **Description** |
| --- | --- |
| Gradient Ascent | Optimization algorithm to find local maximum |
| Gradient Descent | Optimization algorithm to find local minimum |
| Gradient | Rate of change of the function |
| Local Maximum | Maximum value of the function in a neighborhood |
| Machine Learning | Applications in optimization problems, linear regression |
| Data Science | Applications in data analysis, computer vision, NLP |
| Advantages | Simplicity, efficiency |
| Limitations | Getting stuck in local maxima |

## Sources

1. [Gradient Ascent - OpenGenus IQ](https://iq.opengenus.org/gradient-ascent/)
2. [Gradient Ascent: When to use it in machine learning?](https://analyticsindiamag.com/ai-trends/gradient-ascent-when-to-use-it-in-machine-learning/)
3. [PDF](https://web.stanford.edu/class/archive/cs/cs109/cs109.1208/lectures/24_gradient_ascent.pdf)
